In [20]:
import tkinter as tk
from tkinter import ttk, messagebox
from PIL import Image, ImageTk
import torch
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
import pickle
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load models and tokenizers
CATEGORY_MODEL_PATH = r"C:\Users\vidhy\Music\Project\news_category_model"
FAKE_NEWS_MODEL_PATH = r"C:\Users\vidhy\Music\Project\hybrid_model.h5"
TOKENIZER_PATH = r"C:\Users\vidhy\Music\Project\tokenizer.pkl"

category_model = DistilBertForSequenceClassification.from_pretrained(CATEGORY_MODEL_PATH)
category_tokenizer = DistilBertTokenizer.from_pretrained(CATEGORY_MODEL_PATH)
with open(f"{CATEGORY_MODEL_PATH}/label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

fake_news_model = tf.keras.models.load_model(FAKE_NEWS_MODEL_PATH)

with open(TOKENIZER_PATH, "rb") as f:
    tokenizer = pickle.load(f)

# Prediction functions
def predict_combined(news_text):
    sequence = tokenizer.texts_to_sequences([news_text.lower()])
    padded_sequence = pad_sequences(sequence, maxlen=100)
    news_prediction = fake_news_model.predict(padded_sequence)
    authenticity = "Real" if np.argmax(news_prediction, axis=1)[0] == 1 else "Fake"

    encoding = category_tokenizer(news_text, truncation=True, padding=True, max_length=512, return_tensors="pt")
    category_model.eval()
    with torch.no_grad():
        output = category_model(**encoding)
        category_prediction = torch.argmax(output.logits, dim=1).item()
    category = label_encoder.inverse_transform([category_prediction])[0]

    return authenticity, category

# Tkinter Interface
root = tk.Tk()
root.title("News Analyzer")
root.geometry("600x500")

def set_background():
    bg_image = Image.open(r"C:\Users\vidhy\Music\Project\bg.jpg").resize((600, 500))
    bg_photo = ImageTk.PhotoImage(bg_image)
    bg_label = tk.Label(root, image=bg_photo)
    bg_label.image = bg_photo
    bg_label.place(relwidth=1, relheight=1)

def create_styled_button(text, command):
    return tk.Button(root, text=text, command=command, bg="#FFA07A", fg="white", font=("Helvetica", 14), bd=5, relief="raised")

def create_styled_label(text, size=20, color="#3f51b5"):
    return tk.Label(root, text=text, font=("Helvetica", size, "bold"), bg="#ffffff", fg=color, bd=5, relief="flat")

def show_welcome_page():
    for widget in root.winfo_children():
        widget.destroy()
    set_background()

    create_styled_label("Welcome to News Analyzer", 28).pack(pady=20)
    create_styled_label("Analyze news authenticity and category prediction", 16).pack(pady=10)
    create_styled_button("Get Started", show_main_page).pack(pady=20)

def show_main_page():
    for widget in root.winfo_children():
        widget.destroy()
    set_background()

    create_styled_label("Choose a Task", 24, "red").pack(pady=20)
    create_styled_button("Detect News", show_detect_news_page).pack(pady=10)

def show_detect_news_page():
    for widget in root.winfo_children():
        widget.destroy()
    set_background()

    create_styled_label("News Detector", color="red").pack(pady=10)
    text_entry = tk.Text(root, height=10, width=60)
    text_entry.pack(pady=10)

    def predict():
        news_text = text_entry.get("1.0", tk.END).strip()
        if news_text:
            authenticity, category = predict_combined(news_text)
            show_result_page(authenticity, category)

    create_styled_button("Predict", predict).pack(pady=5)
    create_styled_button("Back", show_main_page).pack(pady=5)

def show_result_page(authenticity, category):
    for widget in root.winfo_children():
        widget.destroy()
    set_background()

    create_styled_label("Prediction Result", 24, "green").pack(pady=20)

    if authenticity == "Real":
        result_image = Image.open(r"C:\Users\vidhy\Music\Project\real news.png").resize((200, 200))
    else:
        result_image = Image.open(r"C:\Users\vidhy\Music\Project\fake news .png").resize((200, 200))

    result_photo = ImageTk.PhotoImage(result_image)
    img_label = tk.Label(root, image=result_photo)
    img_label.image = result_photo
    img_label.pack(pady=10)

    result_text = f"Authenticity: {authenticity}\nCategory: {category}"
    create_styled_label(result_text, 18, "green").pack(pady=20)

    create_styled_button("Try Again", show_detect_news_page).pack(pady=10)
    create_styled_button("Exit", root.destroy).pack(pady=5)

set_background()
show_welcome_page()
root.mainloop()

1/1 [==============================] - 0s 40ms/step


In [3]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install transformers


Note: you may need to restart the kernel to use updated packages.
